In [1]:
import torch
from torch.nn import functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np

In [2]:
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)

        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, inputs):
        out1 = F.max_pool2d(F.relu(self.conv1(inputs)), (2, 2))
        out2 = F.max_pool2d(F.relu(self.conv2(out1)), 2)

        out3 = torch.flatten(out2, 1)

        out4 = F.relu(self.fc1(out3))
        out5 = F.relu(self.fc2(out4))
        out6 = self.fc3(out5)

        return out6

In [3]:
net = Net()
net

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [4]:
params = list(net.parameters())
print(len(params))

10


In [6]:
print(params[0].size())

torch.Size([6, 1, 5, 5])


In [8]:
inputs = torch.randn(1, 1, 32, 32)
out = net(inputs)
out

tensor([[-0.0794, -0.0984, -0.0046,  0.0417,  0.0041, -0.0241, -0.1322, -0.0644,
         -0.1039,  0.0510]], grad_fn=<AddmmBackward0>)

In [9]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [10]:
output = net(inputs)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()

loss = criterion(output, target)
loss

tensor(0.3989, grad_fn=<MseLossBackward0>)

In [11]:
print(loss.grad_fn)

In [12]:
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [13]:
net.zero_grad()
print(f"conv1.bas.grad before backward {net.conv1.bias.grad}")

conv1.bas.grad before backward None


In [14]:
loss.backward()

In [15]:
print(f"conv1.bias.grad after backward {net.conv1.bias.grad}")

conv1.bias.grad after backward tensor([-0.0088, -0.0029,  0.0006,  0.0061, -0.0105,  0.0033])
